In [1]:
import gym
from tqdm import tqdm, trange
import torch
import torchbearer
from torch import nn
from torch.nn import functional as F
from torch import optim
import matplotlib.pyplot as plt

env_name = 'MountainCar-v0'
env = gym.make(env_name)
env.seed(1)
torch.manual_seed(1)

In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight, 0, 1)

class NeuralNet(nn.Module) :
    def __init__(self, inp, hidden1, hidden2, out) :
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(inp, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, out)
    def forward(self, x) :
        model = torch.nn.Sequential(
            self.fc1,
            self.fc2,
            self.fc3
        )
        return model(x)

In [3]:
model = NeuralNet(2, 200, 50, 3)

In [5]:
eps = 300
alpha = 0.001
epsilon = 0.3
gamma = 0.99

In [ ]:
steps = 3000
loss_func = nn.MSELoss()
opt = optim.SGD(model.parameters(), alpha)
success = 0

for ep in trange(eps) :
    obs = torch.Tensor(env.reset())
    #Greedy 1st step of episode
    
    for s in range(steps):
        env.render()
        
        # Epsilon greedy
        Q = model(obs)
        e = torch.rand(1).item()
        if e <= epsilon :
            a = torch.randint(env.action_space.n,[1])
            a = a.item()
        else :
            _, a = torch.max(Q,-1)
            a = a.item()
        
        #Q learning update rule
        obs_, reward, done, _ = env.step(a)
        obs_ = torch.Tensor(obs_)
        Q_ = model(obs_)
        Q_max, _ = torch.max(Q_, -1)
        update = reward + gamma * Q_max.detach()
        target = torch.autograd.Variable(Q.clone().data)
        target[a] = update
        
        #GD on appropiate network
        loss = loss_func(Q,target)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if (done and obs_[0] >= 0.5) :
            epsilon *= 0.5
            success += 1
            break
        obs = obs_
print(success, "/300 episodes")

 56%|██████████████████████████████████████████▎                                 | 167/300 [3:13:24<1:50:55, 50.04s/it]